In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, ReLU
from tensorflow.keras.regularizers import l2


In [79]:
# Load the data into a DataFrame
data = pd.read_csv("World_Bank.csv")

# Drop unnecessary columns
df = data.drop(columns=['Country Name', 'Country Code'])

# Check for missing values and handle them (e.g., fill with mean or median)
df.fillna(data.mean(), inplace=True)

scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

data[df.columns[:-1]] = scaler.fit_transform(df[df.columns[:-1]])
data.head()

<ipython-input-79-0f83c7a168b0>:8: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.fillna(data.mean(), inplace=True)


,Country Name,Country Code,Year,ATM Usage,GDP,High Tech Export ($M),High Tech Import,Internet Subscription Per 100,Internet Usage Per 100,IP Income($M),Labour Force (M),Med_High Tech Manufacturing,Mobile Sub Per 100,Personal Remittance ($M),Poverty Count,Stock Traded $B,Tech Cooperation Grant($M),Unemployment Rate,Med High Tech Export
0,Afghanistan,AFG,-1.581139,-1.047724,1.900486,-0.310950,0.267856,-0.888342,-1.512716,-0.287485,-0.314144,-0.885203,-1.665279,-0.336700,-0.391283,-0.335777,0.601381,0.053324,0.38
1,Africa Western and Central,AFW,-1.581139,-1.022006,0.526120,-0.310940,0.294976,-0.880819,-1.398384,-0.281519,-0.035114,-1.088924,-1.254689,-0.002928,-0.391283,-0.308429,0.247003,-0.677386,0.00
2,Albania,ALB,-1.581139,-0.281136,0.566828,-0.310920,-1.329519,-0.616744,-0.065924,-0.287190,-0.326578,-1.076478,-0.226547,-0.312492,-0.367320,-0.335777,-0.082088,1.111529,15.42
3,Algeria,DZA,-1.581139,-0.932582,0.076398,-0.310953,1.573012,-0.700255,-1.212771,-0.287247,-0.304577,-0.873413,-0.251905,-0.336552,-0.385292,-0.335777,-0.006407,0.447075,0.46
4,American Samoa,ASM,-1.581139,0.112893,-0.008894,-0.305228,-0.915259,-0.889094,-0.383512,-0.275140,-0.318194,-0.701790,1.596907,-0.327650,-0.391283,-0.335777,-0.140939,0.123366,0.00


In [17]:
#Split the data into features and target
X = df.drop(columns=['Poverty Count',])
y = df['Poverty Count']

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [74]:
model = Sequential()
model.add(Dense(188, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.01)))
model.add(Dense(132, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dense(128, activation='linear', kernel_regularizer=l2(0.01)))  # Linear activation for regression

# Compile the model with Mean Squared Error (MSE) loss and Adam optimizer
model.compile(optimizer='adam', loss='mean_squared_error')
# Train the model and save the training history
history = model.fit(X_train, y_train, epochs=10, batch_size=25, validation_split=0.2, verbose=1)



# Evaluate the model on training and testing data
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

Epoch 1/10
75/75 [==============================] - 1s 5ms/step - loss: 26769318.0000 - val_loss: 524786.7500
Epoch 2/10
75/75 [==============================] - 0s 4ms/step - loss: 411989.9375 - val_loss: 127532.0859
Epoch 3/10
75/75 [==============================] - 0s 3ms/step - loss: 110754.1016 - val_loss: 7865.7173
Epoch 4/10
75/75 [==============================] - 0s 3ms/step - loss: 2493.5571 - val_loss: 118.1034
Epoch 5/10
75/75 [==============================] - 0s 4ms/step - loss: 114.0518 - val_loss: 105.7720
Epoch 6/10
75/75 [==============================] - 0s 3ms/step - loss: 98.8523 - val_loss: 102.8139
Epoch 7/10
75/75 [==============================] - 0s 4ms/step - loss: 92.1265 - val_loss: 101.5054
Epoch 8/10
75/75 [==============================] - 0s 3ms/step - loss: 89.2645 - val_loss: 100.2560
Epoch 9/10
75/75 [==============================] - 0s 3ms/step - loss: 87.9793 - val_loss: 99.3924
Epoch 10/10
19/19 [==============================] - 0s 1ms/step


In [75]:

train_mse = np.mean((train_pred - y_train.values.reshape(-1, 1)) ** 2)
test_mse = np.mean((test_pred - y_test.values.reshape(-1, 1)) ** 2)

print("Training MSE:", train_mse)
print("Testing MSE:", test_mse)

Training MSE: 86.50282792624863
Testing MSE: 78.34537959693843


In [76]:
model.summary()


Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_94 (Dense)            (None, 188)               3196      
                                                                 
 dense_95 (Dense)            (None, 132)               24948     
                                                                 
 dense_96 (Dense)            (None, 128)               17024     
                                                                 
Total params: 45,168
Trainable params: 45,168
Non-trainable params: 0
_________________________________________________________________


In [77]:
print(data.shape)


(2904, 19)
